# Setup

In [ ]:
!pip install tensorflow keras_cv luketils --upgrade --quiet
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

# Basic Generations

In [ ]:
import keras_cv
from tensorflow import keras
from luketils import visualization
import tensorflow as tf

In [ ]:
keras.mixed_precision.set_global_policy("mixed_float16")
model = keras_cv.models.StableDiffusion(img_width=512, img_height=512, jit_compile=True)

In [ ]:
images = model.text_to_image("", batch_size=3)

In [ ]:
visualization.plot_gallery(images, value_range=(0, 255), rows=1, cols=3)

# Latent Walk

In [ ]:
prompt_1 = "Mysterious ruins in the snow, concept art, digital, artstation"
prompt_2 = "Mysterious ruins in the summer sunlight, concept art, digital, artstation"
interpolation_steps = 50
filename = "ruins_1.gif"

In [ ]:
def export_as_gif(filename, images, frames_per_second=13, rubber_band=False):
    if rubber_band:
        images += images[2:-1][::-1]
    images[0].save(
        filename,
        save_all=True,
        append_images=images[1:],
        duration=1000 // frames_per_second,
        loop=0,
    )

In [ ]:
encoding_1 = tf.squeeze(model.encode_text(prompt_1))
encoding_2 = tf.squeeze(model.encode_text(prompt_2))

batch_size = 10 # This value might to be lowered to 3 on a smaller GPU.
batches = interpolation_steps // batch_size

interpolated_encodings = tf.linspace(encoding_1, encoding_2, interpolation_steps)
batched_encodings = tf.split(interpolated_encodings, batches)

seed = 1337
noise = tf.random.normal((512 // 8, 512 // 8, 4), seed=seed)

images = []
for batch in range(batches):
    images += [
        Image.fromarray(img)
        for img in model.generate_image(
            batched_encodings[batch],
            batch_size=batch_size,
            num_steps=25,
            diffusion_noise=noise,
        )
    ]

export_as_gif(filename, images, rubber_band=True)

![](ruins_1.gif)